In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

c:\Users\heegyukim\Desktop\project\latent-diffusion-lm


In [12]:
from silence_tensorflow import silence_tensorflow
silence_tensorflow()
import torch
from datasets import load_dataset
from src import model_utils
from src.dataset.latent_dialog import LatentDialogDataset
from src.task.latent_dialog import LatentDialogTask
from hydra import initialize, compose
from hydra.core.global_hydra import GlobalHydra

In [8]:
if GlobalHydra.instance().is_initialized():
    GlobalHydra.instance().clear()
initialize("../config/")
config = compose("latent-dialog")
config

{'optimizer': {'cls': 'adam', 'learning_rate': 1e-05}, 'logger': {'name': 'wandb'}, 'dataset': {'train': ['heegyu/aihub_daily_conv_2022_CRF'], 'test': ['heegyu/aihub_daily_conv_2022_CRF']}, 'model': {'model': 'klue/roberta-small', 'autoencoder': 'checkpoint/optimus-v2-44M/optimus-v2-tiny-vae.ckpt', 'latent_dim': 512, 'max_seq_len': 512, 'free_bit': 2.0, 'use_plm': True, 'is_vae': True}, 'trainer': {'train_batch_size': 8, 'train_epochs': 20, 'shuffle': False, 'eval_batch_size': 8, 'eval_strategy': 'epoch', 'project': 'latent-dialog', 'run_name': 'roberta-small-opt44M', 'num_sanity_val_steps': 0, 'limit_train_batches': 110, 'limit_val_batches': 1}}

In [4]:
dataset = LatentDialogDataset(
    ["heegyu/aihub_daily_conv_2022_CRF"]
)

In [6]:
dataset[1]

{'id': 140329,
 'depth': 0,
 'context': '0 : **는 게임 좋아하니?\n1 : 게임 어떤거 좋아해?',
 'response': '2 : 게임 잘 못해서,'}

In [53]:
config.model.max_seq_len = 16
task = LatentDialogTask(config)
task

Some weights of the model checkpoint at klue/roberta-small were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

LatentDialogTask(
  (model): BertVAE(
    (encoder): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(32000, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0): RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
                (dense): Linear(in_features=768, out_fe

In [55]:
task.step(dataset[0:5])


{'id': [140329, 140329, 140329, 140329, 140329], 'depth': [0, 0, 0, 0, 0], 'context': ['0 : **는 게임 좋아하니?', '0 : **는 게임 좋아하니?\n1 : 게임 어떤거 좋아해?', '0 : **는 게임 좋아하니?\n1 : 게임 어떤거 좋아해?\n2 : 게임 잘 못해서,', '0 : **는 게임 좋아하니?\n1 : 게임 어떤거 좋아해?\n2 : 게임 잘 못해서,\n0 : 우린 완전 게임 좋아해', '0 : **는 게임 좋아하니?\n1 : 게임 어떤거 좋아해?\n2 : 게임 잘 못해서,\n0 : 우린 완전 게임 좋아해\n1 : 나는 별로 안 좋아해'], 'response': ['1 : 게임 어떤거 좋아해?', '2 : 게임 잘 못해서,', '0 : 우린 완전 게임 좋아해', '1 : 나는 별로 안 좋아해', '2 : 어떤 게임 주로 하세요?']}
16
16
['[CLS] 0 : * * 는 게임 좋아하니? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD]', '[CLS] * * 는 게임 좋아하니? 1 : 게임 어떤거 좋아해? [SEP]', '[CLS]? 1 : 게임 어떤거 좋아해? 2 : 게임 잘 못해서, [SEP]', '[CLS] 좋아해? 2 : 게임 잘 못해서, 0 : 우린 완전 게임 좋아해 [SEP]', '[CLS], 0 : 우린 완전 게임 좋아해 1 : 나는 별로 안 좋아해 [SEP]']


(tensor(39.3235, grad_fn=<MeanBackward0>),
 Normal(loc: torch.Size([5, 512]), scale: torch.Size([5, 512])))

In [50]:


def pad_to_multiple_of(pad_size: int, multiple_of: int):
    if pad_size % multiple_of == 0:
        return pad_size
    else:
        return multiple_of * ((pad_size // multiple_of) + 1)

pad_to_multiple_of(18, 8)

24